# Phase 2 
Antoine Demers-Bergeron et Joseph Thompson

## Partie 1
### 1. Structure de données pour les composantes connexes
On a implémenté une structure identique aux graphes. On veut que les composantes connexes aient les même propriétés que les graphes.

In [1]:

include("../phase1/node.jl")
include("../phase1/edges.jl")
include("../phase1/graph.jl")
include("../phase1/read_stsp.jl")
include("comp_connexes.jl")
include("../phase1/main.jl")
include("queue.jl")

last_in_first_out_priority_queue

In [2]:
mutable struct Comp_Connexe{Y,T} <: AbstractGraph{T}
    name::String
    nodes::Vector{Node{Y}}
    edges::Vector{Edge{T,Y}}
  end
  
  """Ajoute un noeud à la composante connexe."""
  function add_node!(graph::Comp_Connexe{Y,T}, node::Node{Y}) where {Y,T}
    push!(graph.nodes, node)
    graph
  end
  
  """Ajoute une arête à la composante connexe"""
  function add_edge!(graph::Comp_Connexe{Y,T}, edge::Edge{T,Y}) where {Y,T}
    push!(graph.edges, edge)
    if !(edge.node1.name in nodes_names(graph))
      add_node!(graph, edge.node1)
      @warn "Le noeud",edge.node1.name," n'était pas dans le graphe. Il a été ajouté."
    end
    if !(edge.node2.name in nodes_names(graph))
      add_node!(graph, edge.node2)
      @warn "Le noeud",edge.node2.name," n'était pas dans le graphe. Il a été ajouté."
    end
    graph
  end
  
  """Affiche la composante connexe"""
  function show(graph::Comp_Connexe)
    println("La composante connexe ", name(graph), " a ", nb_nodes(graph), " noeuds.")
    for node in nodes(graph)
      show(node)
    end
    println("La composante connexe ", name(graph), " a ", nb_edges(graph), " arretes.")
    for edge in edges(graph)
      show(edge)
    end
  end

Base.show

### 2. Algorithme de Kruskal


In [3]:
"""
Ce code implémente l'algorithme de Kruskal qui crée un arbre de
recouvrement minimum à partir d'un graphe
"""

""" Tri par insertion une liste d'arêtes selon leur poids """
function insertion!(A::Vector{Edge{Int,Vector{Float64}}}) 
    n = length(A)
    for j = 2 : n
        key = A[j].weight
        edge = A[j]
        i = j - 1
        while i > 0 && A[i].weight > key
            A[i+1] = A[i]
            i = i - 1
        end
        A[i+1] = edge
    end
    A
end

""" Vérifie si les deux noeud d'une arête donnée sont dans la même composante connexe """
function NodesInSameCConnexe(edge::Edge,CConnexes)
    node1, node2 = nodes(edge)
    for C in CConnexes
        if node1 in C.nodes
            if node2 in C.nodes
                return true
            else return false
            end
        end
    end

end

""" Prend une arête [s_i,s_j] et la liste des composantes connexes en input et réunit les composantes connexes de s_i et s_j  """
function Merge_CConnexes!(edge, CConnexes)
    node1, node2 = nodes(edge)
    indice1, indice2 = 0, 0
    for i in eachindex(CConnexes)
        C = CConnexes[i]
        if node1 in nodes(C)  ##Assigne l'indice de la composante connexe du noeud 1
            indice1 = i     
        end
        if node2 in nodes(C)  ####Assigne l'indice de la composante connexe du noeud 2
            indice2 = i
        end
    end

    if indice1 != 0 && indice2 != 0
        append!(CConnexes[indice1].nodes, CConnexes[indice2].nodes)
        append!(CConnexes[indice1].edges, CConnexes[indice2].edges)
        add_edge!(CConnexes[indice1],edge)
        deleteat!(CConnexes, indice2)
    end

    return CConnexes
end

""" Construit l'arbre de recouvrement minimum avec l'algorithme de Kruskal"""
function Kruskal(graphe::Graph)
    A = insertion!(graphe.edges)   ##Tri les arêtes selon leur poids
    CConnexes_list = []
    i=1
    for node in nodes(graphe)   ##Crée une composante connexe pour chaque noeud
        push!(CConnexes_list,Comp_Connexe(string(i),Node{Vector{Float64}}[node],Edge{Int,Vector{Float64}}[]))
    end
    
    for edge in A
        if length(CConnexes_list) == 1
            break       ##On arrête lorsqu'il reste une seule composante connexe
        end
        if (NodesInSameCConnexe(edge,CConnexes_list)) != true    
            CConnexes_list = Merge_CConnexes!(edge,CConnexes_list) ##Réunit les composantes connexes des deux noeuds
        end
    end
    return CConnexes_list[1]
end

Kruskal

On a ensuite testé l'algorithme sur le graphe présenté en exemple dans les notes de laboratoires.

In [4]:
graphe_test = Graph("Test",Node{Vector{Float64}}[],Edge{Int,Vector{Float64}}[])

#Nodes 
nodea = Node("a",[0.])
nodeb = Node("b",[0.])
nodec = Node("c",[0.])
noded = Node("d",[0.])
nodee = Node("e",[0.])
nodef = Node("f",[0.])
nodeg = Node("g",[0.])
nodeh = Node("h",[0.])
nodei = Node("i",[0.])
node_list = [nodea,nodeb,nodec, noded,nodee, nodef, 
                nodeg, nodeh, nodei]

#Edges
edge1 = Edge(nodea,nodeb, 4)
edge2 = Edge(nodea,nodeh, 8)
edge3 = Edge(nodeb, nodeh, 11)
edge4 = Edge(nodeb, nodec, 8)
edge5 = Edge(nodeh, nodei, 7)
edge6 = Edge(nodeh, nodeg, 1)
edge7 = Edge(nodeg, nodei, 6)
edge8 = Edge(nodeg, nodef, 2)
edge9 = Edge(nodec, nodef, 4)
edge10 = Edge(nodei, nodec, 2)
edge11 = Edge(nodef, nodee, 10)
edge12 = Edge(nodee, noded, 9)
edge13 = Edge(nodec,noded, 7)
edge14 = Edge(noded, nodef, 14)
edge_list = [edge1, edge2, edge3, edge4, edge5, edge6,
                edge7, edge8, edge9, edge10, edge11,
                edge12, edge13, edge14]

##Construction du graphe
for i in node_list
    add_node!(graphe_test,i)
end
for i in edge_list
    add_edge!(graphe_test, i)
end

CConnexe = Kruskal(graphe_test)

show(CConnexe)

La composante connexe 1 a 9 noeuds.
Node e, data: [0.0]
Node a, data: [0.0]
Node b, data: [0.0]
Node i, data: [0.0]
Node c, data: [0.0]
Node h, data: [0.0]
Node g, data: [0.0]
Node f, data: [0.0]
Node d, data: [0.0]
La composante connexe 1 a 8 arretes.
Arête de a à b, poids: 4
Arête de i à c, poids: 2
Arête de h à g, poids: 1
Arête de g à f, poids: 2
Arête de c à f, poids: 4
Arête de c à d, poids: 7
Arête de a à h, poids: 8
Arête de e à d, poids: 9


### 3. Test sur diverses instances de TSP symétriques

In [5]:
graphe1, graphe1_nodes = graph_from_tsp("../../instances/stsp/bays29.tsp","graphe1")
CConnexe2 = Kruskal(graphe1)

show(CConnexe2)

Reading of header : ✓
Reading of nodes : ✓
Reading of edges : ✓
La composante connexe 1 a 29 noeuds.
Node 7, data: [1650.0, 650.0]
Node 23, data: [1840.0, 1240.0]
Node 11, data: [840.0, 550.0]
Node 3, data: [40.0, 2090.0]
Node 5, data: [750.0, 2030.0]
Node 9, data: [790.0, 2260.0]
Node 26, data: [490.0, 2130.0]
Node 29, data: [360.0, 1980.0]
Node 6, data: [1030.0, 2070.0]
Node 12, data: [1170.0, 2300.0]
Node 1, data: [1150.0, 1760.0]
Node 28, data: [1260.0, 1910.0]
Node 2, data: [630.0, 1660.0]
Node 21, data: [830.0, 1770.0]
Node 16, data: [1280.0, 1200.0]
Node 8, data: [1490.0, 1630.0]
Node 24, data: [1260.0, 1500.0]
Node 27, data: [1460.0, 1420.0]
Node 4, data: [750.0, 1100.0]
Node 15, data: [750.0, 900.0]
Node 10, data: [710.0, 1310.0]
Node 20, data: [590.0, 1390.0]
Node 14, data: [510.0, 700.0]
Node 18, data: [460.0, 860.0]
Node 22, data: [490.0, 500.0]
Node 19, data: [1040.0, 950.0]
Node 13, data: [970.0, 1340.0]
Node 17, data: [230.0, 590.0]
Node 25, data: [1280.0, 790.0]
La comp

In [6]:
graphe1, graphe1_nodes = graph_from_tsp("../../instances/stsp/swiss42.tsp","graphe1")
CConnexe3 = Kruskal(graphe1)

show(CConnexe3)

Reading of header : ✓
Reading of nodes : ✓
Reading of edges : ✓
La composante connexe 1 a 42 noeuds.
Node 25, data: [0.0, 0.0]
Node 18, data: [0.0, 0.0]
Node 32, data: [0.0, 0.0]
Node 15, data: [0.0, 0.0]
Node 17, data: [0.0, 0.0]
Node 16, data: [0.0, 0.0]
Node 38, data: [0.0, 0.0]
Node 9, data: [0.0, 0.0]
Node 10, data: [0.0, 0.0]
Node 24, data: [0.0, 0.0]
Node 42, data: [0.0, 0.0]
Node 12, data: [0.0, 0.0]
Node 13, data: [0.0, 0.0]
Node 19, data: [0.0, 0.0]
Node 6, data: [0.0, 0.0]
Node 27, data: [0.0, 0.0]
Node 3, data: [0.0, 0.0]
Node 28, data: [0.0, 0.0]
Node 4, data: [0.0, 0.0]
Node 5, data: [0.0, 0.0]
Node 29, data: [0.0, 0.0]
Node 1, data: [0.0, 0.0]
Node 2, data: [0.0, 0.0]
Node 7, data: [0.0, 0.0]
Node 30, data: [0.0, 0.0]
Node 31, data: [0.0, 0.0]
Node 11, data: [0.0, 0.0]
Node 26, data: [0.0, 0.0]
Node 8, data: [0.0, 0.0]
Node 14, data: [0.0, 0.0]
Node 20, data: [0.0, 0.0]
Node 33, data: [0.0, 0.0]
Node 23, data: [0.0, 0.0]
Node 39, data: [0.0, 0.0]
Node 22, data: [0.0, 0.0

## Partie 2 : Implémentation des deux heuristiques



### Implementation du prémiere heuristique

In [9]:
"""Trouve la racine d'un arbre """
function find_root(tree::Tree{T}) where T
  #root is the only node without a parent
  if isnothing(parent(tree))
      return tree
  else
      return find_root(parent(tree))
  end
end


"""Implementation du premier heuristic"""
function rank_union!(tree1::Tree{T}, tree2::Tree{T}) where T
    #trouve la racine de chaque arbre
    root1 = find_root(tree1)
    root2 = find_root(tree2)
    #Cas 1: les racines ont le même rang, on choisit le premier arbre comme racine
    if rank(root1) == rank(root2)
      change_parent!(root2, root1)
      change_rank!(root1, rank(root1) + 1)
      return root1
    #Cas 2: la racine du premier arbre a un rang plus grand que la racine du deuxième arbre
    elseif rank(root1) > rank(root2)
      change_parent!(root2, root1)
      return root1
    #Cas 3: la racine du deuxième arbre a un rang plus grand que la racine du premier arbre
    else
      change_parent!(root1, root2)
      return root2
    end
end

rank_union!

In [10]:
tree_A = Tree("A", 0)
tree_B = Tree("B", 1)
new_tree = rank_union!(tree_A, tree_B)
tree_C = Tree("C", 2)
tree_D = Tree("D", 3)
second_tree = rank_union!(tree_C, tree_D)
final_tree = rank_union!(new_tree, second_tree)
show(final_tree)


Node A has  rank 2 and 2 children.
node_visited: B     
 its parent is A     
 its rank is 0
node_visited: C     
 its parent is A     
 its rank is 1
   its children are: 
     D
node_visited: D     
 its parent is C     
 its rank is 0


### Preuve que le rang d'un arbre avec le premier algorithme heuristique est toujours inférieur ou égal à |S|-1 et qu'il est également toujours inférieur ou égal à floor(log2(|S|)).

**Preuve par contradiction que le rang est toujours inférieur ou égal à |S|-1**

Pour prouver que le rang est toujours inférieur ou égal à |S|-1, supposons qu'il existe un graphe généré par la première heuristique avec un rang de |S| ou plus. Nous savons que la seule façon d'augmenter le rang est de le fusionner avec un arbre de même rang. Étant donné que le rang commence à zéro, cela signifie qu'il a dû être fusionné au moins |S|+1 fois. Il ne peut pas être fusionné plus de |S| fois car il n'y a pas assez de nœuds. C'est une contradiction.

**Preuve par induction que le rang est toujours inférieur ou égal à floor(log2(|S|))**

*Cas de base, fusion de deux graphes de rang zéro.*

Supposons que nous commençons notre algorithme avec deux graphes disjoints de rang zéro. Lorsque nous les fusionnons, leur nombre total de nœuds est |S| $\geq$ 2. Le nouveau rang serait 1. Alors nous savons que rang(G) = 1 $\leq$ floor(log2(|S|)).

*Étape d'induction : fusion de deux arbres A et B ayant rang(A) $\leq$ floor(log2(|$S_A$| )) et rang(B) $\leq$ floor(log2(|$S_B$| ))*

Supposons que nous ayons créé deux arbres disjoints, A et B, qui respectent rang(G) $\leq$ floor(log2(|S|)). supposons que $S_A$ représente les nœuds de A, $S_B$ les nœuds de B, et $S$ les nœuds de leur union. Lorsque nous les combinons, il y a trois possibilités :

1. rang(A) > rang(B)
2. rang(B) > rang(A)
3. rang(A) = rang(B)

Pour les cas 1 et 2, nous constatons que le rang(A $\cup$ B) = max(rang(A), rang(B)) et |S| = |$S_A$| + |$S_B$|. Étant donné que rang(A) $\leq$ floor(log2(|$S_A$| )) et rang(B) $\leq$ floor(log2(|$S_B$|)), et à la fois floor(log2(|$S_A$| )) et floor(log2(|$S_B$| )) sont inférieurs à floor(log2(|$S_A$ + $S_B$| )), alors rang(A $\cup$ B) < floor(log2(|$S_A$ + $S_B$| )).

Pour le troisième cas, supposons que rang(A)=rang(B). Ensuite, A devient le parent et rang(A $\cup$ B) = rang (A) + 1. À partir de nos hypothèses, nous savons que |$S_A$|, |$S_B$| $\geq$ $2^{rang(A)}$ nœuds. Donc |S| = |$S_A$| + |$S_B$| $\geq$ 2 * $2^{rang(A)}$. 
Ensuite, floor(log2(|S|)) $\geq$ floor(log2(|2 * $2^{rang(A)}$) = 1 + rang(A). Par conséquent, le rang(A $\cup$ B) = rang(A) + 1 $\leq$ floor(log2(|S|)).

### Second heuristique

la fonction find_root() a été modifiée pour qu'elle change le parent de tous les nœuds du chemin de recherche vers la racine.

In [11]:

"""Implementation du deuxieme heuristic"""
function path_compression!(tree::Tree{T}) where T
    #la racine est le seul noeud sans parent
    if isnothing(parent(tree))
        return tree
    else
        root =  path_compression!(parent(tree))
        change_parent!(tree, root)
        return root
    end
end



path_compression!

voici un test de l'heuristique. D'abord commencons avec un petit arbre. 

(Notez que le rang n'est utilisé que pour la première heuristique et pour stocker la longueur de la plus courte arête reliant l'arête au graphe dans l'algorithme de Prim. Sinon, il sera toujours nul.)

In [12]:
tree_root = Tree("1", 0)
child1 = Tree("2", 0)
child2 = Tree("3", 0)
child3 = Tree("4", 0)
child4 = Tree("5",0)
child5 = Tree("6",0)
child6 = Tree("7",0)
first_row = [ child1, child2, child3]
second_row = [child4, child6]
for child in first_row
    change_parent!(child, tree_root)
end
for child in second_row
    change_parent!(child, child3)
end
change_parent!(child5, child4)
show(tree_root)

Node 1 has  rank 0 and 3 children.
node_visited: 2     
 its parent is 1     
 its rank is 0
node_visited: 3     
 its parent is 1     
 its rank is 0
node_visited: 4     
 its parent is 1     
 its rank is 0
   its children are: 
     5
     7
node_visited: 5     
 its parent is 4     
 its rank is 0
   its children are: 
     6
node_visited: 7     
 its parent is 4     
 its rank is 0
node_visited: 6     
 its parent is 5     
 its rank is 0


Maintenant on utilise l'heuristique. Comme prévu, tous les nœuds de l'arbre sur le chemin vers le nœud racine voient leur parent modifié en nœud racine. Le nœud 7 reste inchangé car il n'était pas dans le chemin de recherche.

In [13]:
path_compression!(child5)
show(tree_root)

Node 1 has  rank 0 and 5 children.
node_visited: 2     
 its parent is 1     
 its rank is 0
node_visited: 3     
 its parent is 1     
 its rank is 0
node_visited: 4     
 its parent is 1     
 its rank is 0
   its children are: 
     7
node_visited: 5     
 its parent is 1     
 its rank is 0
node_visited: 6     
 its parent is 1     
 its rank is 0
node_visited: 7     
 its parent is 4     
 its rank is 0


## Partie 3 : Algorithme de Prim

### Implementation

Pour mettre en œuvre cet algorithme, nous utilisons une structure de données de file d'attente prioritaire. Cela était basé sur la structure de données de file d'attente prioritaire qui était donnée dans les notes de cours. Dans notre cas, notre file d'attente prioritaire est un tableau d'éléments prioritaires, composé de l'élément d'origine et de la priorité, un nombre. Les détails de la file d'attente prioritaire peuvent être consultés dans l'annexe.

Les performances de l'algorithme pourraient être améliorées si la file d'attente prioritaire était implémentée sous forme de tas binaire. En raison de contraintes de temps, nous avons dû la laisser sous forme de tableau.

 
 
L'algorithme commence par convertir les objets de nœuds du graphe d'origine en objets d'arbre, et leur attribue une priorité. 0 pour le nœud de départ, et l'infini pour les autres. Nous générons ensuite un dictionnaire d'adjacence qui est un dictionnaire de dictionnaires. La première clé est chaque nœud dans le graphe. La deuxième clé est chaque nœud qui est adjacent au premier nœud. Les valeurs sont la distance au voisin.

Après avoir généré le dictionnaire d'adjacence, nous entrons dans la boucle principale. Nous prenons le nœud ayant la valeur la plus basse dans la file d'attente prioritaire et l'ajoutons à notre arbre. Nous mettons ensuite à jour les distances des voisins de ce nœud et les valeurs de la file d'attente prioritaire. L'algorithme se termine lorsque nous avons ajouté chaque nœud à l'arbre.

Malheureusement, nous avons rencontré des difficultés avec l'utilisation du mot-clé "in" en Julia. Pour une raison quelconque, il disait parfois qu'un élément était dans la file d'attente prioritaire alors qu'il ne l'était pas. Nous recherchons donc manuellement la file d'attente prioritaire. Étant donné que nous parcourons chaque nœud dans la file d'attente prioritaire, vérifions chacun de ses voisins, puis vérifions s'ils sont également dans la file d'attente prioritaire, les performances de notre algorithme sont O(|V|^3). Les performances pourraient être améliorées si nous pouvions utiliser "in" et implémenter un tas binaire pour la file d'attente prioritaire.

In [14]:

"""crée une liste d'adjacence a partir d'un graphe"""
function adjacency_dict( graph::Graph , priority_queue::PriorityQueue{PriorityItem{Tree{T}}}) where T
    #Dictionaire de adjacence
    adj_dict = Dict()
    #Dictionaire de correspondance entre les noeuds et les items de la file de priorite
    correspondance_dict = Dict()
    #Chaque noeud est une clef du dictionaire de adjacence
    for (node, priority_item) in zip(nodes(graph),priority_queue.items)
        adj_dict[priority_item] = Dict()
        correspondance_dict[node] = priority_item
    end
    #Ajoute les voisins de chaque noeud dans le dictionaire de adjacence
    for edge in edges(graph)
        node1, node2 = nodes(edge)
        prior_item1 = correspondance_dict[node1]
        prior_item2 = correspondance_dict[node2]
        adj_dict[prior_item1][prior_item2] = weight(edge)
        adj_dict[prior_item2][prior_item1] = weight(edge)
    end
    return adj_dict
end

"""Crée une file de priorite des arbres à partir d'un graphe. Le noeud de depart a une priorite de 0 et les autres ont une priorite de Inf"""
function prims_priority_queue(graph::Graph{Y,T}, start_node_name::String) where {Y,T}
    priority_queue = PriorityQueue{PriorityItem{Tree{Y}}}()
    for node in nodes(graph)
        if name(node) == start_node_name
            blank_tree = Tree(name(node), data(node))
            priority_item = PriorityItem( 0 , blank_tree)
        else
            blank_tree = Tree(name(node), data(node))
            priority_item = PriorityItem( Inf, blank_tree)
        end
        push!(priority_queue, priority_item)
    end
    return priority_queue
end

"""Implementation de l'algorithme de Prim"""
function prims_algorithm(graph::Graph{Y,T}; start_node_name::Any = nothing) where {Y,T}
    #initialisation
    if isnothing(start_node_name)
        start_node_name = name(nodes(graph)[1])
    end
    #initialisation de la file de priorite et du dictionaire d'adjacence
    priority_queue = prims_priority_queue(graph, start_node_name)
    adjacency_list = adjacency_dict(graph, priority_queue)
    #sauvegarde de la racine
    root = poplast!(priority_queue)
    priority_node = root
    #boucle principale
    while !is_empty(priority_queue)
        for  (neighbor, edge_weight) in adjacency_list[priority_node]
            #Met a jour la priorite du voisin si elle est dans le priority_que et elle est plus petite que la priorite actuelle
            for item in priority_queue.items
                if name(data(neighbor)) == name(data(item))
                    if  edge_weight < priority(neighbor)
                        change_parent!(data(neighbor), data(priority_node))
                        change_rank!(data(neighbor), edge_weight)
                        priority!(neighbor, edge_weight)
                    end
                break
                end
            end
        end
        #prend le noeud avec la plus petite priorite
        priority_node = poplast!(priority_queue)         
    end
    return data(root)
end

prims_algorithm

Afin de faciliter la comparaison entre l'algorithme de Prim et celui de Kruskal, nous avons mis en œuvre cette fonction auxiliaire pour convertir une structure de données arborescente en un graphe.

In [15]:
"""En utilisant BFS, Convert un arbre en graphe. """
function tree_to_graph(tree::Tree{T}) where T
  graph = Graph(name(tree), Node{T}[], Edge{Float64, T}[])
  #On commence avec les enfants de la racine. La racine sera ajouté tel que parent de ses enfants
  nodes_to_visit = copy(children(tree))
  while length(nodes_to_visit) != 0
    current_tree = popfirst!(nodes_to_visit)
    node = Node(name(current_tree), data(current_tree))
    parent_tree= parent(current_tree)
    for child in children(current_tree)
      push!(nodes_to_visit, child)
    end
    if !isnothing(parent_tree)
      parent_node = Node(name(parent_tree), data(parent_tree))
      distance = convert(Float64, rank(current_tree))
      edge = Edge(parent_node, node, distance)
      add_edge!(graph, edge, safe = false)
    end
  end
  graph
end

tree_to_graph

### Evaluation

Ci-dessous, nous testons l'algorithme sur le graphe d'exemple du laboratoire. Il donne les mêmes résultats que ceux de Kruskal.

In [16]:

test_tree = prims_algorithm(graphe_test,start_node_name =  "a")
test_graph = tree_to_graph(test_tree)
show(test_graph)

Graph a has 9 nodes.
Node a, data: [0.0]
Node b, data: [0.0]
Node c, data: [0.0]
Node f, data: [0.0]
Node d, data: [0.0]
Node i, data: [0.0]
Node g, data: [0.0]
Node e, data: [0.0]
Node h, data: [0.0]
Graph a has 8 edges.
Arête de a à b, poids: 4.0
Arête de b à c, poids: 8.0
Arête de c à f, poids: 4.0
Arête de c à d, poids: 7.0
Arête de c à i, poids: 2.0
Arête de f à g, poids: 2.0
Arête de d à e, poids: 9.0
Arête de g à h, poids: 1.0


Voici une fonction qui additionne les arêtes d'un arbre de couverture minimale. Les deux algorithmes devraient avoir la même valeur finale.

In [17]:
#affiche la somme des poids des arretes d'un graphe
function sum_of_weights(graph::AbstractGraph{T}) where T
    sum = 0
    for edge in edges(graph)
        sum += weight(edge)
    end
    return sum
end

sum_of_weights (generic function with 1 method)

Nous testons ceci sur l'instance bays29. Nous constatons que nous obtenons des solutions légèrement différentes de celles de l'algorithme de Kruskal, mais qu'ils ont tous deux la même valeur finale de l'objectif.

In [18]:
graphe1, graphe1_nodes = graph_from_tsp("../../instances/stsp/bays29.tsp","graphe1")
println("running Prim's algorithm")
@time bays_29_tree = prims_algorithm(graphe1)
bays_29_graph = tree_to_graph(bays_29_tree)

Reading of header : ✓
Reading of nodes : ✓
Reading of edges : ✓
running Prim's algorithm
  0.015350 seconds (11.87 k allocations: 705.439 KiB, 95.13% compilation time)


Graph{Vector{Float64}, Float64}("1", Node{Vector{Float64}}[Node{Vector{Float64}}("1", [1150.0, 1760.0]), Node{Vector{Float64}}("28", [1260.0, 1910.0]), Node{Vector{Float64}}("21", [830.0, 1770.0]), Node{Vector{Float64}}("6", [1030.0, 2070.0]), Node{Vector{Float64}}("2", [630.0, 1660.0]), Node{Vector{Float64}}("12", [1170.0, 2300.0]), Node{Vector{Float64}}("5", [750.0, 2030.0]), Node{Vector{Float64}}("20", [590.0, 1390.0]), Node{Vector{Float64}}("26", [490.0, 2130.0]), Node{Vector{Float64}}("9", [790.0, 2260.0])  …  Node{Vector{Float64}}("25", [1280.0, 790.0]), Node{Vector{Float64}}("16", [1280.0, 1200.0]), Node{Vector{Float64}}("14", [510.0, 700.0]), Node{Vector{Float64}}("7", [1650.0, 650.0]), Node{Vector{Float64}}("27", [1460.0, 1420.0]), Node{Vector{Float64}}("17", [230.0, 590.0]), Node{Vector{Float64}}("22", [490.0, 500.0]), Node{Vector{Float64}}("23", [1840.0, 1240.0]), Node{Vector{Float64}}("24", [1260.0, 1500.0]), Node{Vector{Float64}}("8", [1490.0, 1630.0])], Edge{Float64, Vect

Voici le graphe de l'arbre crée avec Prim's algorithm

In [19]:
show(bays_29_graph)

Graph 1 has 29 nodes.
Node 1, data: [1150.0, 1760.0]
Node 28, data: [1260.0, 1910.0]
Node 21, data: [830.0, 1770.0]
Node 6, data: [1030.0, 2070.0]
Node 2, data: [630.0, 1660.0]
Node 12, data: [1170.0, 2300.0]
Node 5, data: [750.0, 2030.0]
Node 20, data: [590.0, 1390.0]
Node 26, data: [490.0, 2130.0]
Node 9, data: [790.0, 2260.0]
Node 10, data: [710.0, 1310.0]
Node 29, data: [360.0, 1980.0]
Node 4, data: [750.0, 1100.0]
Node 13, data: [970.0, 1340.0]
Node 3, data: [40.0, 2090.0]
Node 15, data: [750.0, 900.0]
Node 11, data: [840.0, 550.0]
Node 19, data: [1040.0, 950.0]
Node 18, data: [460.0, 860.0]
Node 25, data: [1280.0, 790.0]
Node 16, data: [1280.0, 1200.0]
Node 14, data: [510.0, 700.0]
Node 7, data: [1650.0, 650.0]
Node 27, data: [1460.0, 1420.0]
Node 17, data: [230.0, 590.0]
Node 22, data: [490.0, 500.0]
Node 23, data: [1840.0, 1240.0]
Node 24, data: [1260.0, 1500.0]
Node 8, data: [1490.0, 1630.0]
Graph 1 has 28 edges.
Arête de 1 à 28, poids: 45.0
Arête de 1 à 21, poids: 65.0
Arête 

Les deux algorithmes donnent la même valeur de la fonction objectif.

In [20]:
println(sum_of_weights(bays_29_graph))
println(sum_of_weights(CConnexe2))

1557.0
1557


Nous nous assurons également qu'ils obtiennent les mêmes résultats pour l'instance swiss42.

In [21]:
# graphe1, graphe1_nodes = graph_from_tsp("../../instances/stsp/swiss42.tsp","graphe1")
# CConnexe = Kruskal(graphe1)
swiss_42_graph, swiss_42_nodes = graph_from_tsp("../../instances/stsp/swiss42.tsp","graphe1")
println("running Prim's algorithm on swiss42")
@time swiss_42_tree = prims_algorithm(swiss_42_graph)
swiss_42_graph = tree_to_graph(swiss_42_tree)
println("Prim's algorithm total distance ", sum_of_weights(swiss_42_graph))
println("Kruskal's algorithm total distance ", sum_of_weights(CConnexe3))

Reading of header : ✓
Reading of nodes : ✓
Reading of edges : ✓
running Prim's algorithm on swiss42
  0.001584 seconds (8.57 k allocations: 376.500 KiB)


Prim's algorithm total distance 1079.0
Kruskal's algorithm total distance 1079


## Annexe 1 implementation du PriorityQueue

Voici notre implémentation de la file d'attente prioritaire. Elle est basée sur les notes de cours.

Son parent est une file d'attente abstraite. La file d'attente prioritaire est un Vector de PriorityItems. Chaque PriorityItem contient les données que nous voulons prioriser et sa priorité, un nombre. Pour l'algorithme de Prim, nous disons que le plus petit nombre devrait passer en premier, nous avons donc dû mettre en œuvre minimum() et poplast().

In [22]:
import Base.length, Base.push!, Base.popfirst!
import Base.show
import Base.isless, Base.==, Base.maximum, Base.minimum

"""Type abstrait dont d'autres types de files dériveront."""
abstract type AbstractQueue{T} end

"""Ajoute `item` à la fin de la file `s`."""
function push!(q::AbstractQueue{T}, item::T) where T
    push!(q.items, item)
    q
end

"""Retire et renvoie l'objet du début de la file."""
popfirst!(q::AbstractQueue) = popfirst!(q.items)

"""Indique si la file est vide."""
is_empty(q::AbstractQueue) = length(q.items) == 0

"""Donne le nombre d'éléments sur la file."""
length(q::AbstractQueue) = length(q.items)

"""Affiche une file."""
show(q::AbstractQueue) = show(q.items)

"""Type abstrait dont d'autres types d'éléments de file de priorité dériveront."""
abstract type AbstractPriorityItem{T} end

#PRIORITY QUEUE
"""Type représentant un élément de la file de priorité. Il contient:
 un élément de type T
 une priorité de type Int."""
mutable struct PriorityItem{T} <: AbstractPriorityItem{T} 
    priority::Real
    data::T
end

"""Crée un élément de la file de priorité."""
function PriorityItem(priority::Real, data::T) where T 
    PriorityItem{T}(max(0, priority), data)
end

"""Renvoie les données d'un élément de la file de priorité."""
data(p::PriorityItem) = p.data

"""Renvoie la priorité de un élément de la file de priorité."""
priority(p::PriorityItem) = p.priority

"""change la priorité de un élément de la file de priorité."""
function priority!(p::PriorityItem, priority::Real) 
    p.priority = max(0, priority)
    p

end
"""compare la priorité de deux éléments de la file de priorité."""
isless(p1::PriorityItem, p2::PriorityItem) = p1.priority < p2.priority

==(p1::PriorityItem, p2::PriorityItem) = p1.priority == p2.priority 

"""File de priorité."""
mutable struct PriorityQueue{T <: AbstractPriorityItem} <: AbstractQueue{T}
    items::Vector{T}
end

"""initialisation d'une file de priorité"""
PriorityQueue{T}() where T = PriorityQueue(T[])

"""Crée un vector de noms des éléments de la file de priorité."""
function names(q::PriorityQueue)
    names = []
    for item in q.items
        push!(names, item.data.name)
    end
    names
end

"""Renvoie les élements de la file de priorité."""
items(q::PriorityQueue) = q.items

maximum(q::AbstractQueue) = maximum(q.items)

minimum(q::AbstractQueue) = minimum(q.items)

"""Retire et renvoie l'élément ayant la plus haute priorité."""
function popfirst!(q::PriorityQueue)
    highest = maximum(q)
    idx = findall(x -> x == highest, q.items)[1]
    deleteat!(q.items, idx)
    highest
end

"""Retire et renvoie l'élément ayant la plus base priorité."""
function poplast!(q::PriorityQueue)
    lowest = minimum(q)
    idx = findall(x -> x == lowest, q.items)[1]
    deleteat!(q.items, idx)
    lowest
end

"""imprime une file de priorité."""
function show(q::PriorityQueue)
    for item in q.items
        println("item: $(item.data) priority: $(item.priority)")
    end
end




Base.show